### Regularization Exercises


In [38]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.preprocessing import StandardScaler
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize , poly)
from functools import partial
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from ISLP.models import (Stepwise, sklearn_selected, sklearn_selection_path)
from l0bnb import fit_path
import matplotlib.pyplot as plt


In [ ]:
np.random.seed(0)
rng = np.random.default_rng()
sim_data = rng.normal(0,1,100)
noise = rng.normal(0,1,100)

beta_0 = 1
beta_1 = 2
beta_2 = 3
beta_3 = 4

response = beta_0 + beta_1*sim_data + beta_2*(sim_data**2) + beta_3*(sim_data**3) + noise
response

$C_p = \frac{RSS + 2P\sigma^2}{n}$

In [96]:
# Forward Selection using C_p, AIC, BIC
# Backward Selection
# Lasso with Cross validation


power_range = np.arange(0,11)
predictors = np.power.outer(sim_data, power_range)
data = pd.DataFrame(predictors, columns = [f'X^{i}' for i in power_range])

design = MS(data).fit(data)
x = design.transform(data)
sigma2 = sm.OLS(response,x).fit().scale

def C_p(sigma2,estimator, x, y) : 
    n, p = x.shape
    Yhat = estimator.predict(x)
    rss = np.sum((y - Yhat)**2)
    return -(rss + 2*p*sigma2)/n

def AIC(sigma, estimator, x, y) : 
    n, p = x.shape
    Yhat = estimator.predict(x)
    rss = np.sum((y - Yhat)**2)
    log_likelihood = - (n / 2) * np.log(2 * np.pi * sigma) - (rss / (2 * sigma))
    aic = -2 * (log_likelihood /n)
    return (aic) + 2*p

def BIC(sigma, estimator, x, y) : 
    n, p = x.shape
    Yhat = estimator.predict(x)
    rss = np.sum((y - Yhat)**2)
    log_likelihood = - (n / 2) * np.log(2 * np.pi * sigma) - (rss / (2 * sigma))
    bic = -2 * (log_likelihood / n)
    return (bic ) + (np.log(n) * p)

neg_cp = partial(C_p, sigma2)
aic = partial(AIC, sigma2)
bic = partial(BIC, sigma2)

# Forward Selection
forward = Stepwise.first_peak(design, direction="forward", max_terms=len(design.terms))
backward = Stepwise.first_peak(design, direction = "backwards", max_terms=len(design.terms))
forward_sel = sklearn_selected(sm.OLS, forward, scoring = neg_cp)
backward_sel = sklearn_selected(sm.OLS, backward, scoring = neg_cp)

forward_aic = sklearn_selected(sm.OLS, forward, scoring = aic)
forward_bic = sklearn_selected(sm.OLS, forward, scoring = bic)
backward_aic = sklearn_selected(sm.OLS, backward, scoring = aic)
backward_bic = sklearn_selected(sm.OLS, backward, scoring = bic)

forward_sel.fit(x, response)
backward_sel.fit(x, response)
forward_aic.fit(x, response)
forward_bic.fit(x, response)
backward_aic.fit(x, response)
backward_bic.fit(x, response)

print(f"Forward Selection : {forward_sel.selected_state_}")
print(f"Backward Selection : {backward_sel.selected_state_}")
print(f"Forward Selection AIC : {forward_aic.selected_state_}")
print(f"Forward Selection BIC : {forward_bic.selected_state_}")
print(f"Backward Selection AIC : {backward_aic.selected_state_}")
print(f"Backward Selection BIC : {backward_bic.selected_state_}")


Forward Selection : ('X^1', 'X^2', 'X^3')
Backward Selection : ('X^1', 'X^2', 'X^3')
Forward Selection AIC : ('X^0', 'X^4')
Forward Selection BIC : ('X^0', 'X^4')
Backward Selection AIC : ('X^0', 'X^4')
Backward Selection BIC : ('X^0', 'X^4')
